In [8]:
import pandas as pd
from tdmsdata_edited import TdmsData
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import curve_fit   
import os
import importlib
# import doppler_shift_2024


In [9]:
folder_path = './power120/'

In [10]:
def read_tdms(file, channel):
    TDMS = TdmsData(''.join([folder_path, file]))
    raw_data = TDMS.get_raw_data(''.join(['Channel ', str(channel)]))  # 2D np array
    return raw_data

In [11]:
def create_df(raw_data):
    return pd.DataFrame({'Cycle No.': raw_data[:, 0], 'Time (s)': raw_data[:, 1], 
                         'Laser Frequency (THz)': raw_data[:, 2], 'Power (mW)': raw_data[:, 3], 
                         'Approx Time': raw_data[:, 4], 'SDUMP': raw_data[:, 5]})

In [12]:
def process_tdms(raw_df):
    step_size = 0.001 # nm 
    bin_width_thz = 0.00008 
    # bins = 50

    # wavelengths_raw = raw_df['Laser Frequency (THz)']
    # wavelengths_raw = pd.to_numeric(wavelengths_raw, errors='coerce')
    # wavelengths_raw = wavelengths_raw.dropna()

    # # counts_df = pd.DataFrame({'Freq': wavelengths_raw})
    # # counts_df = wavelengths_raw.value_counts().reset_index()
    # # counts_df.columns = ['Freq', 'Count']
        
    # # counts_df = counts_df.sort_values(by='Freq', ascending=True).reset_index()

    # raw_df['Freq bin'] = pd.cut(raw_df['Laser Frequency (THz)'], bins, right=False)
    # # raw_df = raw_df[raw_df['Freq bin'] >= 400]

    # binned_df = raw_df.groupby('Freq bin', observed=True).size().reset_index(name='Count')
    # binned_df['Bin center'] = binned_df['Freq bin'].apply(lambda x: x.mid)
    # binned_df = binned_df[binned_df['Count'] > 0]
    # binned_df.columns = ['Freq bin', 'Count', 'Bin center']

    df = raw_df.copy()
    # df = df[df['Time (s)'] <= 90].reset_index(drop=True)
    df['Laser Frequency (THz)'] = pd.to_numeric(df['Laser Frequency (THz)'], errors='coerce')
    df = df.dropna(subset=['Laser Frequency (THz)'])
    df = df[df['Laser Frequency (THz)'] > 423.349].reset_index(drop=True)

    fmin = df['Laser Frequency (THz)'].min()
    fmax = df['Laser Frequency (THz)'].max()
    start = np.floor(fmin / bin_width_thz) * bin_width_thz
    stop  = np.ceil(fmax / bin_width_thz) * bin_width_thz + bin_width_thz
    bins = np.arange(start, stop + 0.5*bin_width_thz, bin_width_thz)

    df['Freq bin'] = pd.cut(df['Laser Frequency (THz)'], bins, right=False)  
    binned_df = (df
                 .groupby('Freq bin', observed=True)
                 .size()
                 .reset_index(name='Count'))

    # binned_df = binned_df[binned_df['Count'] > 0]
    binned_df['Bin center'] = binned_df['Freq bin'].apply(lambda iv: iv.mid)


    return binned_df

In [13]:
def get_scatter_plot(df, file):
    # freqs = counts_df['Bin center'].cat.categories.astype(float)
    # # freqs = counts_df['Freq']
    # counts = counts_df['Count']

    df['Bin center'] = pd.to_numeric(df['Bin center'], errors='coerce')
    df['Count'] = pd.to_numeric(df['Count'], errors='coerce')
    df = df.replace([np.inf, -np.inf], np.nan).dropna(subset=['Bin center','Count'])
    df = df.sort_values('Bin center')

    # plt.xlim(423.349, 423.354)
    # plt.ylim(-10, 10000)

    x = df['Bin center'].to_numpy()
    y = df['Count'].to_numpy()
    max_counts = df['Count'].max()

    # plt binned counts 
    plt.plot(x,y/max_counts, label = file)
    plt.xlabel('Freq THz')
    plt.ylabel('Count')
    plt.title(f'{file}')
    # plt.legend()

    # plt.show()


In [14]:
for filename in os.listdir(folder_path):
# filename = '20251030_1103_Sn_120_set2_ref4.tdms'
# filename = '20251030_1052_Sn_120_set2_ref3.tdms'
# filename = '20251030_1043_Sn_120_set2_ref2.tdms'
# filename = '20251030_1034_Sn_120_set2_ref1.tdms'
    if filename.endswith('.tdms'):

    #     if '120' in filename:
    #         isotope = 120
    #         # freq_range = (195.0938, 195.0955)
    #         freq_range = None
    #     elif '122' in filename:
    #         isotope = 122
    #     elif '124' in filename:
    #         isotope = 124
    #         # freq_range = (195.096, 195.098)
    #     elif '116' in filename:
    #         isotope = 116
    #     #     freq_range = (195.091, 195.0925)
    #     elif '118' in filename:
    #         isotope = 118
    #     #     freq_range= (195.0925, 195.094)
    #     elif '112' in filename: 
    #         isotope = 112

        # print(filename)

        raw_data = read_tdms(filename, channel=2)
        raw_df = create_df(raw_data)
        counts_df = process_tdms(raw_df)

        get_scatter_plot(counts_df, filename)

UnboundLocalError: local variable 'data' referenced before assignment